# Notebook 00: End-to-End Pipeline for a Single Setup

**Purpose:** This notebook demonstrates how to run the entire experimental pipeline (Hyperparameter Optimization, Final Training, and Evaluation) for a *single, specific setup* by calling the core functions defined in `run_hpo.py`, `run_final_training.py`, and `run_evaluation.py`.

This allows for:
1.  Testing the full workflow programmatically.
2.  Running a specific setup if modifications are needed without using the command line for all scripts.
3.  Understanding how the different stages connect and pass information (e.g., best HPO params to final training).

**Setup to Run in this Notebook:** `resnet_mid_noaug`

In [1]:
import torch
import os
import time # For potential timing if desired, though scripts handle internal timing

# Import shared utilities and configurations
import utils
import config # Contains HPO_CONFIG_LIST, UNFREEZE_MAPS, output dirs, defaults

# Import the core processing functions from our scripts
from run_hpo import perform_hpo_for_setup
from run_final_training import perform_final_training_for_setup
from run_evaluation import perform_evaluation_for_setup

# --- Define the Single Setup to Run ---
SETUP_ID_TO_RUN = 'resnet_mid_noaug'
print(f"### Will run pipeline for Setup ID: {SETUP_ID_TO_RUN} ###")

# --- Global Parameters for this Run (can override config defaults if needed) ---
# These would typically come from argparse in the scripts, here we set them directly
# Or, we can rely on defaults set within config.py and the script functions
HPO_EPOCHS = config.DEFAULT_HPO_EPOCHS
FINAL_TRAIN_EPOCHS = config.DEFAULT_FINAL_TRAIN_EPOCHS
PATIENCE = config.DEFAULT_PATIENCE
BATCH_SIZE = config.DEFAULT_BATCH_SIZE # Using one batch size for simplicity here, scripts might allow separate

# --- Device Setup ---
if torch.backends.mps.is_available() and torch.backends.mps.is_built():
    device = torch.device("mps")
elif torch.cuda.is_available():
    device = torch.device("cuda")
else:
    device = torch.device("cpu")
print(f"Using device: {device}")

# --- Ensure Output Directories Exist ---
config.create_output_dirs()

### Will run pipeline for Setup ID: resnet_mid_noaug ###
Using device: mps
Output directories ensured.


## Stage 1: Hyperparameter Optimization

Running HPO for the selected setup (`SETUP_ID_TO_RUN`).
This will iterate through the hyperparameter grid defined in `config.HPO_CONFIG_LIST`, run short training trials, and save the best parameters found and the full HPO history to the `results/hpo/` directory.

In [ ]:
print(f"\n{'='*5} STAGE 1: Hyperparameter Optimization {'='*5}")
hpo_start_time = time.time()

# Call the HPO function from run_hpo.py
# It will handle parsing the setup_id internally if designed to do so,
# or we can pass the components if the function expects them directly.
# Assuming perform_hpo_for_setup takes setup_id and uses config for maps.

best_hpo_params_found = perform_hpo_for_setup(
    setup_id=SETUP_ID_TO_RUN,
    hpo_grid=config.HPO_CONFIG_LIST,
    num_epochs_hpo=HPO_EPOCHS,
    batch_size=BATCH_SIZE,
    device=device
    # The function saves its outputs to files within config.HPO_DIR
)

hpo_end_time = time.time()
print(f"\n{'*'*10} HPO Stage for {SETUP_ID_TO_RUN} finished in {(hpo_end_time - hpo_start_time)/60:.2f} minutes {'*'*10}")

if best_hpo_params_found:
    print(f"Best HPO parameters found: {best_hpo_params_found}")
    # These are also saved to results/hpo/best_params_{SETUP_ID_TO_RUN}.json
else:
    print(f"HPO did not complete successfully or find best parameters for {SETUP_ID_TO_RUN}. Cannot proceed.")
    # Consider raising an error or stopping the notebook here if HPO is critical for next steps

## Stage 2: Final Model Training

Running final training for `SETUP_ID_TO_RUN` using the best hyperparameters found in Stage 1.
This loads `results/hpo/best_params_{SETUP_ID_TO_RUN}.json`, trains for more epochs with early stopping on training loss, and saves the best model and training history to `results/final_training/`.

In [ ]:
if best_hpo_params_found: # Proceed only if HPO was successful
    print(f"\n{'='*20} STAGE 2: Final Model Training {'='*20}")
    final_train_start_time = time.time()

    # Call the final training function from run_final_training.py
    # It will load the best_params file itself based on setup_id.
    saved_model_filepath = perform_final_training_for_setup(
        setup_id=SETUP_ID_TO_RUN,
        max_epochs=FINAL_TRAIN_EPOCHS,
        patience=PATIENCE,
        batch_size=BATCH_SIZE,
        device=device
        # The function saves its outputs to files within config.FINAL_TRAINING_DIR
    )

    final_train_end_time = time.time()
    print(f"\n{'*'*10} Final Training Stage for {SETUP_ID_TO_RUN} finished in {(final_train_end_time - final_train_start_time)/60:.2f} minutes {'*'*10}")

    if saved_model_filepath:
        print(f"Best model saved to: {saved_model_filepath}")
        # This path is also saved internally by the function to results/final_training/best_model_{SETUP_ID_TO_RUN}.pth
    else:
        print(f"Final training did not complete successfully or model was not saved for {SETUP_ID_TO_RUN}. Cannot proceed to evaluation.")
else:
    print("\nSkipping Stage 2: Final Model Training due to HPO failure or no best parameters found.")
    saved_model_filepath = None # Ensure it's None if HPO failed

## Stage 3: Evaluation

Evaluating the best model from Stage 2 for `SETUP_ID_TO_RUN` on the test set.
This loads `results/final_training/best_model_{SETUP_ID_TO_RUN}.pth`, generates predictions, calculates metrics, and saves predictions and metrics to `results/evaluation/`.

In [ ]:
if saved_model_filepath and os.path.exists(saved_model_filepath): # Proceed only if final training produced a model
    print(f"\n{'='*20} STAGE 3: Evaluation {'='*20}")
    eval_start_time = time.time()

    # Call the evaluation function from run_evaluation.py
    # It will load the model file itself based on setup_id and config.FINAL_TRAINING_DIR
    perform_evaluation_for_setup(
        setup_id=SETUP_ID_TO_RUN,
        device=device
        # The function saves its outputs to files within config.EVALUATION_DIR
    )

    eval_end_time = time.time()
    print(f"\n{'*'*10} Evaluation Stage for {SETUP_ID_TO_RUN} finished in {(eval_end_time - eval_start_time)/60:.2f} minutes {'*'*10}")
    print(f"\nEvaluation results (predictions and metrics CSVs) are saved in: {config.EVALUATION_DIR}")
else:
    print("\nSkipping Stage 3: Evaluation due to final training failure or model not found.")

## Pipeline Complete for `resnet_mid_noaug`

The HPO, Final Training, and Evaluation stages have been executed for the setup `resnet_mid_noaug`.

**Outputs can be found in the `results/` directory:**
*   `results/hpo/hpo_history_resnet_mid_noaug.json`
*   `results/hpo/best_params_resnet_mid_noaug.json`
*   `results/final_training/best_model_resnet_mid_noaug.pth`
*   `results/final_training/final_training_history_resnet_mid_noaug.json`
*   `results/evaluation/predictions_resnet_mid_noaug.csv`
*   `results/evaluation/eval_metrics_resnet_mid_noaug.csv`